In [1]:
#import all packages
import geopandas as gpd
import pandas as pd

In [2]:
# read the file into a geodataframe
landslide_data = gpd.read_file('a0000000a.gdbtable')

In [3]:
landslide_data.head()

,SLIDE_TYPE,M_WIDTH,S_LENGTH,SLOPE,COVER,YEAR_1,HEADELEV,TAILELEV,ELE_DIFF,GULLY,CLASS,ENTLI_NO,EASTING,NORTHING,Shape_Length,geometry
0,R,10.5,16.0,44.0,C,1964,136.0,127.0,9.0,N,A2,02NED0001E,829062.359007,842154.704576,16.246627,"MULTILINESTRING ((829062.359 842154.705, 82904..."
1,R,11.5,14.0,29.0,C,1964,122.0,113.0,9.0,N,C1,02NED0002E,829176.865680,842509.807641,13.765049,"MULTILINESTRING ((829176.866 842509.808, 82917..."
2,R,10.0,16.0,22.0,C,1964,48.0,39.0,9.0,N,A2,02NED0003E,827401.227700,842230.475928,16.192822,"MULTILINESTRING ((827401.228 842230.476, 82738..."
3,R,11.0,22.5,22.0,C,1964,53.0,42.0,11.0,N,C1,02NED0004E,827407.309973,842208.930039,22.423121,"MULTILINESTRING ((827407.310 842208.930, 82738..."
4,R,12.5,14.0,43.0,C,1964,97.0,87.0,10.0,N,B1,02NED0005E,827526.113452,842190.282812,13.861750,"MULTILINESTRING ((827526.113 842190.283, 82753..."


In [4]:
landslide_data.shape

(111569, 16)

In [5]:
#convert file to csv to use in PowerBi or Google Studio
#data.to_csv('landslide2.csv')

In [6]:
# dataset information
landslide_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 111569 entries, 0 to 111568
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   SLIDE_TYPE    111569 non-null  object  
 1   M_WIDTH       111569 non-null  float64 
 2   S_LENGTH      111569 non-null  float64 
 3   SLOPE         111569 non-null  float64 
 4   COVER         111569 non-null  object  
 5   YEAR_1        111569 non-null  object  
 6   HEADELEV      111569 non-null  float64 
 7   TAILELEV      111569 non-null  float64 
 8   ELE_DIFF      111569 non-null  float64 
 9   GULLY         111569 non-null  object  
 10  CLASS         111569 non-null  object  
 11  ENTLI_NO      111569 non-null  object  
 12  EASTING       111569 non-null  float64 
 13  NORTHING      111569 non-null  float64 
 14  Shape_Length  111569 non-null  float64 
 15  geometry      111569 non-null  geometry
dtypes: float64(9), geometry(1), object(6)
memory usage: 13.6+ MB


In [7]:
landslide_data.isnull().sum()

SLIDE_TYPE      0
M_WIDTH         0
S_LENGTH        0
SLOPE           0
COVER           0
YEAR_1          0
HEADELEV        0
TAILELEV        0
ELE_DIFF        0
GULLY           0
CLASS           0
ENTLI_NO        0
EASTING         0
NORTHING        0
Shape_Length    0
geometry        0
dtype: int64

In [9]:
landslide_data.duplicated().sum()

0